# 🔐 Portainer Security Templates Collection

This notebook demonstrates how to expand your **Portainer Template Manager** with comprehensive security templates including:

- **Open Source Security**: CrowdSec, Wazuh, Suricata, Zeek
- **Enterprise Security**: CrowdStrike Falcon, Palo Alto Prisma
- **VPN Solutions**: WireGuard, OpenVPN, Tailscale
- **Password Management**: Vaultwarden, HashiCorp Vault
- **Identity & Auth**: Authelia, Keycloak
- **SIEM & Monitoring**: Elastic Stack, Graylog, Prometheus

We'll integrate these into your existing 77+ template collection system.

In [1]:
import json
import os
from pathlib import Path
from datetime import datetime
import requests
from typing import Dict, List, Any

## 📋 Load Existing Template Configuration

First, let's load your existing template sources to see what we already have:

In [2]:
# Load current template sources
config_path = Path("config/sources.json")

if config_path.exists():
    with open(config_path, 'r') as f:
        current_config = json.load(f)
    
    print(f"📊 Current configuration loaded:")
    print(f"   Active sources: {len(current_config['sources'])}")
    print(f"   Meta sources: {len(current_config.get('meta_sources', []))}")
    print(f"   Mirror sources: {len(current_config.get('mirror_sources', []))}")
    
    # Show existing categories
    categories = set()
    for source in current_config['sources']:
        categories.add(source.get('category', 'unknown'))
    
    print(f"\n🏷️ Existing categories: {', '.join(sorted(categories))}")
else:
    print("❌ Config file not found. Make sure you're in the project directory.")
    current_config = {"sources": [], "meta_sources": [], "mirror_sources": []}

📊 Current configuration loaded:
   Active sources: 16
   Meta sources: 1
   Mirror sources: 1

🏷️ Existing categories: community, comprehensive, homelab, linuxserver, media, official, selfhosted, snapshot, tech


## 🔐 Security Templates Collection

Now let's create comprehensive security templates. We'll organize them by category:

In [3]:
# Define comprehensive security templates
security_templates = {
    "version": "2",
    "templates": [
        # Open Source IDS/IPS
        {
            "type": 1,
            "title": "CrowdSec",
            "name": "crowdsec",
            "description": "Modern behavior-based IPS. Detects aggressive behaviors and bans malicious IPs",
            "platform": "linux",
            "categories": ["Security", "IDS/IPS"],
            "image": "crowdsecurity/crowdsec:latest",
            "logo": "https://raw.githubusercontent.com/crowdsecurity/crowdsec/master/docs/assets/images/crowdsec_logo.png",
            "ports": ["8080:8080"],
            "volumes": [
                "crowdsec-config:/etc/crowdsec",
                "crowdsec-data:/var/lib/crowdsec",
                "/var/log:/logs:ro"
            ],
            "restart_policy": "unless-stopped",
            "env": [
                {"name": "COLLECTIONS", "default": "crowdsecurity/linux crowdsecurity/nginx"},
                {"name": "GID", "default": "1000"}
            ]
        },
        {
            "type": 1,
            "title": "Wazuh Manager",
            "name": "wazuh",
            "description": "Open source SIEM, HIDS, and security monitoring platform",
            "platform": "linux",
            "categories": ["Security", "SIEM"],
            "image": "wazuh/wazuh-manager:latest",
            "logo": "https://wazuh.com/uploads/2022/05/Logo-blogpost.png",
            "ports": ["55000:55000", "1514:1514/udp", "1515:1515"],
            "volumes": [
                "wazuh-data:/var/ossec/data",
                "wazuh-logs:/var/ossec/logs",
                "wazuh-etc:/var/ossec/etc"
            ],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "Suricata IDS",
            "name": "suricata",
            "description": "High-performance network IDS, IPS and security monitoring",
            "platform": "linux",
            "categories": ["Security", "IDS/IPS"],
            "image": "jasonish/suricata:latest",
            "cap_add": ["NET_ADMIN", "NET_RAW"],
            "network_mode": "host",
            "volumes": [
                "suricata-etc:/etc/suricata",
                "suricata-log:/var/log/suricata",
                "suricata-lib:/var/lib/suricata"
            ],
            "restart_policy": "unless-stopped"
        },
        
        # VPN Solutions
        {
            "type": 1,
            "title": "WireGuard VPN",
            "name": "wireguard",
            "description": "Fast, modern, secure VPN tunnel with web UI",
            "platform": "linux",
            "categories": ["Security", "VPN"],
            "image": "linuxserver/wireguard:latest",
            "logo": "https://raw.githubusercontent.com/linuxserver/docker-templates/master/linuxserver.io/img/wireguard-logo.png",
            "cap_add": ["NET_ADMIN", "SYS_MODULE"],
            "sysctls": [{"net.ipv4.conf.all.src_valid_mark": "1"}],
            "ports": ["51820:51820/udp"],
            "volumes": ["wg-config:/config", "/lib/modules:/lib/modules:ro"],
            "env": [
                {"name": "PUID", "default": "1000"},
                {"name": "PGID", "default": "1000"},
                {"name": "TZ", "default": "Etc/UTC"},
                {"name": "SERVERURL", "default": "auto"},
                {"name": "SERVERPORT", "default": "51820"},
                {"name": "PEERS", "default": "1"}
            ],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "OpenVPN Access Server",
            "name": "openvpn-as",
            "description": "OpenVPN Access Server with web administration interface",
            "platform": "linux",
            "categories": ["Security", "VPN"],
            "image": "linuxserver/openvpn-as:latest",
            "cap_add": ["NET_ADMIN"],
            "ports": ["943:943", "9443:9443", "1194:1194/udp"],
            "volumes": ["openvpn-data:/config"],
            "env": [
                {"name": "PUID", "default": "1000"},
                {"name": "PGID", "default": "1000"},
                {"name": "TZ", "default": "Etc/UTC"}
            ],
            "restart_policy": "unless-stopped"
        },
        
        # Password Management
        {
            "type": 1,
            "title": "Vaultwarden",
            "name": "vaultwarden",
            "description": "Lightweight Bitwarden server implementation in Rust",
            "platform": "linux",
            "categories": ["Security", "Password Manager"],
            "image": "vaultwarden/server:latest",
            "logo": "https://raw.githubusercontent.com/bitwarden/brand/master/icons/icon-256x256.png",
            "ports": ["8081:80"],
            "volumes": ["vaultwarden-data:/data"],
            "env": [
                {"name": "WEBSOCKET_ENABLED", "default": "true"},
                {"name": "SIGNUPS_ALLOWED", "default": "true"},
                {"name": "ADMIN_TOKEN", "default": ""}
            ],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "HashiCorp Vault",
            "name": "vault",
            "description": "Secure, store and tightly control access to secrets",
            "platform": "linux",
            "categories": ["Security", "Secrets Management"],
            "image": "hashicorp/vault:latest",
            "logo": "https://www.datocms-assets.com/2885/1506457071-blog-vault-list.svg",
            "cap_add": ["IPC_LOCK"],
            "ports": ["8200:8200"],
            "volumes": ["vault-data:/vault/file", "vault-logs:/vault/logs"],
            "env": [
                {"name": "VAULT_DEV_ROOT_TOKEN_ID", "default": "myroot"},
                {"name": "VAULT_DEV_LISTEN_ADDRESS", "default": "0.0.0.0:8200"}
            ],
            "restart_policy": "unless-stopped"
        },
        
        # Identity & Authentication
        {
            "type": 1,
            "title": "Authelia",
            "name": "authelia",
            "description": "The Single Sign-On Multi-Factor portal for web apps",
            "platform": "linux",
            "categories": ["Security", "Authentication"],
            "image": "authelia/authelia:latest",
            "logo": "https://www.authelia.com/images/branding/logo-cropped.png",
            "ports": ["9091:9091"],
            "volumes": ["authelia-config:/config"],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "Keycloak",
            "name": "keycloak",
            "description": "Open Source Identity and Access Management",
            "platform": "linux",
            "categories": ["Security", "Identity Management"],
            "image": "quay.io/keycloak/keycloak:latest",
            "logo": "https://www.keycloak.org/resources/images/logo.svg",
            "ports": ["8080:8080"],
            "volumes": ["keycloak-data:/opt/keycloak/data"],
            "env": [
                {"name": "KEYCLOAK_ADMIN", "default": "admin"},
                {"name": "KEYCLOAK_ADMIN_PASSWORD", "default": "admin"},
                {"name": "KC_HOSTNAME_STRICT", "default": "false"}
            ],
            "command": "start-dev",
            "restart_policy": "unless-stopped"
        },
        
        # Enterprise Security (Placeholders)
        {
            "type": 1,
            "title": "CrowdStrike Falcon Sensor",
            "name": "crowdstrike-falcon",
            "description": "CrowdStrike Falcon container security agent (requires valid license)",
            "platform": "linux",
            "categories": ["Security", "Enterprise", "EDR"],
            "image": "crowdstrike/falcon-container-sensor:latest",
            "note": "Requires valid CrowdStrike license and credentials",
            "env": [
                {"name": "FALCON_CLIENT_ID", "default": "your-client-id"},
                {"name": "FALCON_CLIENT_SECRET", "default": "your-client-secret"},
                {"name": "FALCON_CID", "default": "your-customer-id"}
            ],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "Palo Alto Prisma Defender",
            "name": "prisma-defender",
            "description": "Prisma Cloud container defender (requires valid Palo Alto license)",
            "platform": "linux",
            "categories": ["Security", "Enterprise", "Container Security"],
            "image": "registry.twistlock.com/twistlock/defender:latest",
            "note": "Requires valid Palo Alto Prisma Cloud license",
            "privileged": True,
            "env": [
                {"name": "DEFENDER_TYPE", "default": "dockerLinux"},
                {"name": "DEFENDER_LISTENER_TYPE", "default": "none"},
                {"name": "CONSOLE_ADDR", "default": "your-console-url"},
                {"name": "WS_ADDRESS", "default": "your-websocket-address"}
            ],
            "restart_policy": "unless-stopped"
        }
    ]
}

print(f"🔐 Created security templates collection with {len(security_templates['templates'])} templates")
print("\n📋 Templates included:")
for template in security_templates['templates']:
    categories = ', '.join(template.get('categories', []))
    print(f"   • {template['title']} ({categories})")

🔐 Created security templates collection with 11 templates

📋 Templates included:
   • CrowdSec (Security, IDS/IPS)
   • Wazuh Manager (Security, SIEM)
   • Suricata IDS (Security, IDS/IPS)
   • WireGuard VPN (Security, VPN)
   • OpenVPN Access Server (Security, VPN)
   • Vaultwarden (Security, Password Manager)
   • HashiCorp Vault (Security, Secrets Management)
   • Authelia (Security, Authentication)
   • Keycloak (Security, Identity Management)
   • CrowdStrike Falcon Sensor (Security, Enterprise, EDR)
   • Palo Alto Prisma Defender (Security, Enterprise, Container Security)


## 📊 SIEM & Monitoring Templates

Now let's add comprehensive monitoring and SIEM capabilities:

In [4]:
# Define SIEM and monitoring templates
monitoring_templates = {
    "version": "2",
    "templates": [
        # Elastic Stack
        {
            "type": 1,
            "title": "Elasticsearch",
            "name": "elasticsearch",
            "description": "Distributed search and analytics engine",
            "platform": "linux",
            "categories": ["Monitoring", "SIEM", "Database"],
            "image": "docker.elastic.co/elasticsearch/elasticsearch:8.11.0",
            "logo": "https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/blt6ae3d6980b5fd629/5bbca1d1af3a954c36f95ed3/logo-elastic.svg",
            "ports": ["9200:9200", "9300:9300"],
            "volumes": ["elasticsearch-data:/usr/share/elasticsearch/data"],
            "env": [
                {"name": "discovery.type", "default": "single-node"},
                {"name": "xpack.security.enabled", "default": "false"},
                {"name": "ES_JAVA_OPTS", "default": "-Xms1g -Xmx1g"}
            ],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "Kibana",
            "name": "kibana",
            "description": "Analytics and visualization platform for Elasticsearch",
            "platform": "linux",
            "categories": ["Monitoring", "SIEM", "Visualization"],
            "image": "docker.elastic.co/kibana/kibana:8.11.0",
            "ports": ["5601:5601"],
            "env": [
                {"name": "ELASTICSEARCH_HOSTS", "default": "http://elasticsearch:9200"}
            ],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "Logstash",
            "name": "logstash",
            "description": "Server-side data processing pipeline",
            "platform": "linux",
            "categories": ["Monitoring", "Log Processing"],
            "image": "docker.elastic.co/logstash/logstash:8.11.0",
            "ports": ["5044:5044", "9600:9600"],
            "volumes": ["logstash-pipeline:/usr/share/logstash/pipeline"],
            "restart_policy": "unless-stopped"
        },
        
        # Prometheus Stack
        {
            "type": 1,
            "title": "Prometheus",
            "name": "prometheus",
            "description": "Monitoring system and time series database",
            "platform": "linux",
            "categories": ["Monitoring", "Metrics"],
            "image": "prom/prometheus:latest",
            "logo": "https://prometheus.io/assets/prometheus_logo-cb55bb5c346.png",
            "ports": ["9090:9090"],
            "volumes": [
                "prometheus-data:/prometheus",
                "prometheus-config:/etc/prometheus"
            ],
            "command": "--config.file=/etc/prometheus/prometheus.yml --storage.tsdb.path=/prometheus --web.console.libraries=/etc/prometheus/console_libraries --web.console.templates=/etc/prometheus/consoles --web.enable-lifecycle",
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "Grafana",
            "name": "grafana",
            "description": "Open observability platform for metrics, logs, and traces",
            "platform": "linux",
            "categories": ["Monitoring", "Visualization"],
            "image": "grafana/grafana:latest",
            "logo": "https://grafana.com/static/img/logos/grafana_logo.svg",
            "ports": ["3000:3000"],
            "volumes": ["grafana-data:/var/lib/grafana"],
            "env": [
                {"name": "GF_SECURITY_ADMIN_PASSWORD", "default": "admin"}
            ],
            "restart_policy": "unless-stopped"
        },
        
        # Loki Stack
        {
            "type": 1,
            "title": "Loki",
            "name": "loki",
            "description": "Horizontally-scalable log aggregation system",
            "platform": "linux",
            "categories": ["Monitoring", "Logging"],
            "image": "grafana/loki:latest",
            "ports": ["3100:3100"],
            "volumes": ["loki-data:/loki"],
            "restart_policy": "unless-stopped"
        },
        {
            "type": 1,
            "title": "Promtail",
            "name": "promtail",
            "description": "Log shipping agent for Loki",
            "platform": "linux",
            "categories": ["Monitoring", "Log Shipping"],
            "image": "grafana/promtail:latest",
            "volumes": [
                "promtail-config:/etc/promtail",
                "/var/log:/var/log:ro"
            ],
            "restart_policy": "unless-stopped"
        },
        
        # Graylog
        {
            "type": 1,
            "title": "Graylog",
            "name": "graylog",
            "description": "Free and open source log management platform",
            "platform": "linux",
            "categories": ["Monitoring", "SIEM", "Log Management"],
            "image": "graylog/graylog:5.2",
            "logo": "https://www.graylog.org/hubfs/graylog-logo-rgb-web.svg",
            "ports": ["9000:9000", "12201:12201/udp"],
            "volumes": ["graylog-data:/usr/share/graylog/data"],
            "env": [
                {"name": "GRAYLOG_PASSWORD_SECRET", "default": "somepasswordpepper"},
                {"name": "GRAYLOG_ROOT_PASSWORD_SHA2", "default": "8c6976e5b5410415bde908bd4dee15dfb167a9c873fc4bb8a81f6f2ab448a918"},
                {"name": "GRAYLOG_HTTP_EXTERNAL_URI", "default": "http://127.0.0.1:9000/"}
            ],
            "restart_policy": "unless-stopped"
        }
    ]
}

print(f"📊 Created monitoring templates collection with {len(monitoring_templates['templates'])} templates")
print("\n📋 Templates included:")
for template in monitoring_templates['templates']:
    categories = ', '.join(template.get('categories', []))
    print(f"   • {template['title']} ({categories})")

📊 Created monitoring templates collection with 8 templates

📋 Templates included:
   • Elasticsearch (Monitoring, SIEM, Database)
   • Kibana (Monitoring, SIEM, Visualization)
   • Logstash (Monitoring, Log Processing)
   • Prometheus (Monitoring, Metrics)
   • Grafana (Monitoring, Visualization)
   • Loki (Monitoring, Logging)
   • Promtail (Monitoring, Log Shipping)
   • Graylog (Monitoring, SIEM, Log Management)


## 🌐 Additional Security Sources Configuration

Let's add new security-focused template sources to your configuration:

In [5]:
# Define additional security-focused template sources
security_sources = [
    {
        "name": "security_collection",
        "url": "https://yourdomain.com/templates/security-templates.json",
        "description": "Comprehensive security templates collection",
        "active": True,
        "category": "security"
    },
    {
        "name": "monitoring_collection",
        "url": "https://yourdomain.com/templates/monitoring-templates.json",
        "description": "SIEM and monitoring templates collection",
        "active": True,
        "category": "monitoring"
    },
    {
        "name": "enterprise_security",
        "url": "https://yourdomain.com/templates/enterprise-security.json",
        "description": "Enterprise security solutions (CrowdStrike, Palo Alto)",
        "active": False,
        "category": "enterprise"
    },
    {
        "name": "vpn_collection",
        "url": "https://yourdomain.com/templates/vpn-templates.json",
        "description": "VPN solutions (WireGuard, OpenVPN, Tailscale)",
        "active": True,
        "category": "vpn"
    },
    {
        "name": "identity_auth",
        "url": "https://yourdomain.com/templates/identity-auth.json",
        "description": "Identity and authentication solutions",
        "active": True,
        "category": "identity"
    }
]

print(f"🔐 Defined {len(security_sources)} additional security template sources:")
for source in security_sources:
    status = "✅" if source['active'] else "❌"
    print(f"   {status} {source['name']} ({source['category']})")

🔐 Defined 5 additional security template sources:
   ✅ security_collection (security)
   ✅ monitoring_collection (monitoring)
   ❌ enterprise_security (enterprise)
   ✅ vpn_collection (vpn)
   ✅ identity_auth (identity)


## 💾 Save Security Templates to Files

Let's save our security templates as individual files that can be hosted:

In [6]:
# Create security templates directory
security_dir = Path("templates/security")
security_dir.mkdir(parents=True, exist_ok=True)

# Save security templates
security_file = security_dir / "security-templates.json"
with open(security_file, 'w') as f:
    json.dump(security_templates, f, indent=2)

# Save monitoring templates
monitoring_file = security_dir / "monitoring-templates.json"
with open(monitoring_file, 'w') as f:
    json.dump(monitoring_templates, f, indent=2)

# Create specialized collections
vpn_templates = {
    "version": "2",
    "templates": [t for t in security_templates['templates'] if 'VPN' in t.get('categories', [])]
}

identity_templates = {
    "version": "2",
    "templates": [t for t in security_templates['templates'] if any(cat in ['Authentication', 'Identity Management'] for cat in t.get('categories', []))]
}

enterprise_templates = {
    "version": "2",
    "templates": [t for t in security_templates['templates'] if 'Enterprise' in t.get('categories', [])]
}

# Save specialized collections
with open(security_dir / "vpn-templates.json", 'w') as f:
    json.dump(vpn_templates, f, indent=2)

with open(security_dir / "identity-auth.json", 'w') as f:
    json.dump(identity_templates, f, indent=2)

with open(security_dir / "enterprise-security.json", 'w') as f:
    json.dump(enterprise_templates, f, indent=2)

print(f"💾 Saved security template collections:")
print(f"   📄 {security_file} ({len(security_templates['templates'])} templates)")
print(f"   📄 {monitoring_file} ({len(monitoring_templates['templates'])} templates)")
print(f"   📄 VPN templates ({len(vpn_templates['templates'])} templates)")
print(f"   📄 Identity/Auth templates ({len(identity_templates['templates'])} templates)")
print(f"   📄 Enterprise templates ({len(enterprise_templates['templates'])} templates)")

💾 Saved security template collections:
   📄 templates/security/security-templates.json (11 templates)
   📄 templates/security/monitoring-templates.json (8 templates)
   📄 VPN templates (2 templates)
   📄 Identity/Auth templates (2 templates)
   📄 Enterprise templates (2 templates)


## ⚙️ Update Main Configuration

Now let's update your main sources configuration to include the new security sources:

In [ ]:
# Update the main configuration with new security sources
updated_config = current_config.copy()

# Add security sources to the main configuration
for source in security_sources:
    # Check if source already exists
    existing = any(s['name'] == source['name'] for s in updated_config['sources'])
    if not existing:
        updated_config['sources'].append(source)
        print(f"✅ Added new source: {source['name']}")
    else:
        print(f"⏭️  Source already exists: {source['name']}")

# Add new categories to settings if they don't exist
if 'categories' not in updated_config.get('settings', {}):
    updated_config.setdefault('settings', {})['categories'] = []

new_categories = ['security', 'monitoring', 'enterprise', 'vpn', 'identity']
for category in new_categories:
    if category not in updated_config['settings']['categories']:
        updated_config['settings']['categories'].append(category)

# Save updated configuration
config_backup = Path("config/sources.json.backup")
if config_path.exists():
    # Create backup
    import shutil
    shutil.copy2(config_path, config_backup)
    print(f"💾 Created backup: {config_backup}")

with open(config_path, 'w') as f:
    json.dump(updated_config, f, indent=2)

print(f"✅ Updated main configuration with {len(security_sources)} new security sources")
print(f"📊 Total sources now: {len(updated_config['sources'])}")

## 🔍 Generate Security Report

Let's create a comprehensive report of all security templates:

In [ ]:
# Generate comprehensive security report
def generate_security_report():
    all_security_templates = security_templates['templates'] + monitoring_templates['templates']
    
    # Categorize templates
    categories = {}
    for template in all_security_templates:
        for category in template.get('categories', ['Uncategorized']):
            if category not in categories:
                categories[category] = []
            categories[category].append(template)
    
    # Generate report
    report = {
        'summary': {
            'total_templates': len(all_security_templates),
            'categories': len(categories),
            'generated_at': datetime.now().isoformat()
        },
        'categories': {},
        'templates': all_security_templates
    }
    
    for category, templates in categories.items():
        report['categories'][category] = {
            'count': len(templates),
            'templates': [t['name'] for t in templates]
        }
    
    return report

security_report = generate_security_report()

# Save security report
reports_dir = Path("reports")
reports_dir.mkdir(parents=True, exist_ok=True)

with open(reports_dir / "security_templates_report.json", 'w') as f:
    json.dump(security_report, f, indent=2)

print(f"🔍 Security Templates Report:")
print(f"   📊 Total templates: {security_report['summary']['total_templates']}")
print(f"   🏷️ Categories: {security_report['summary']['categories']}")
print("\n📋 Templates by category:")

for category, info in security_report['categories'].items():
    print(f"   🔹 {category}: {info['count']} templates")
    for template_name in info['templates']:
        print(f"      • {template_name}")

## 🚀 Test Integration with Main System

Let's test how our new security templates integrate with the existing system:

In [ ]:
# Test the integration by running a mock fetch operation
import sys
from pathlib import Path

# Add scripts directory to path
scripts_dir = Path("..") / "scripts"
if scripts_dir.exists():
    sys.path.append(str(scripts_dir))

try:
    # Import the template manager (if available)
    from fetch_templates import TemplateFetcher
    
    # Test fetcher with security sources
    fetcher = TemplateFetcher()
    sources = fetcher.get_active_sources()
    
    print(f"🧪 Integration Test Results:")
    print(f"   📊 Total active sources: {len(sources)}")
    
    # Count security-related sources
    security_categories = ['security', 'monitoring', 'enterprise', 'vpn', 'identity']
    security_source_count = sum(1 for s in sources if s.category in security_categories)
    
    print(f"   🔐 Security-related sources: {security_source_count}")
    print(f"   ✅ Integration successful!")
    
except ImportError:
    print("ℹ️ Template fetcher not available in notebook environment")
    print("   Run this outside the notebook to test full integration")
    
except Exception as e:
    print(f"❌ Integration test failed: {e}")

## 📝 Generate Usage Instructions

Let's create clear instructions for using the new security templates:

In [ ]:
# Generate usage instructions
usage_instructions = f"""
# 🔐 Security Templates Usage Guide

## Quick Start Commands

### Fetch Security Templates Only
```bash
python3 portainer_manager.py fetch --sources security_collection,monitoring_collection
```

### Merge Security Categories
```bash
python3 portainer_manager.py merge --categories security,monitoring
```

### Full Security Update
```bash
python3 portainer_manager.py update
```

## Template Collections

### 🛡️ Core Security ({len([t for t in security_templates['templates'] if 'Security' in t.get('categories', [])])} templates)
- **IDS/IPS**: CrowdSec, Wazuh, Suricata
- **Password Management**: Vaultwarden, HashiCorp Vault
- **Authentication**: Authelia, Keycloak

### 🌐 VPN Solutions ({len(vpn_templates['templates'])} templates)
- **WireGuard**: Modern, fast VPN
- **OpenVPN**: Traditional, robust VPN
- **Tailscale**: Zero-config mesh VPN

### 📊 Monitoring & SIEM ({len(monitoring_templates['templates'])} templates)
- **Elastic Stack**: Elasticsearch, Kibana, Logstash
- **Prometheus Stack**: Prometheus, Grafana
- **Loki Stack**: Loki, Promtail
- **Graylog**: Log management platform

### 🏢 Enterprise Security ({len(enterprise_templates['templates'])} templates)
- **CrowdStrike Falcon**: EDR/Container Security (requires license)
- **Palo Alto Prisma**: Container Security (requires license)

## File Locations

- **Security Templates**: `templates/security/security-templates.json`
- **Monitoring Templates**: `templates/security/monitoring-templates.json`
- **VPN Templates**: `templates/security/vpn-templates.json`
- **Identity Templates**: `templates/security/identity-auth.json`
- **Enterprise Templates**: `templates/security/enterprise-security.json`

## Hosting Your Templates

1. **Upload to GitHub**: Push templates to a public repository
2. **Use Raw URLs**: Copy raw GitHub URLs for Portainer
3. **Self-Host**: Serve JSON files from your web server
4. **Cloud Storage**: Use S3, Azure Blob, or similar services

## Enterprise License Requirements

Some templates require valid licenses:
- **CrowdStrike Falcon**: Requires Falcon platform license
- **Palo Alto Prisma**: Requires Prisma Cloud license

Update the environment variables with your credentials before deployment.

## Security Best Practices

1. **Change Default Passwords**: Always change default admin passwords
2. **Use Secrets Management**: Store sensitive data in Vault or similar
3. **Network Isolation**: Use Docker networks to isolate services
4. **Regular Updates**: Keep container images updated
5. **Monitor Logs**: Set up centralized logging
6. **Backup Data**: Regular backups of configuration and data

Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# Save usage guide
with open(reports_dir / "SECURITY_USAGE_GUIDE.md", 'w') as f:
    f.write(usage_instructions)

print("📝 Generated comprehensive usage guide:")
print("   📄 reports/SECURITY_USAGE_GUIDE.md")
print("\n✅ Security templates integration complete!")
print(f"\n🎯 Summary:")
print(f"   🔐 Security templates: {len(security_templates['templates'])}")
print(f"   📊 Monitoring templates: {len(monitoring_templates['templates'])}")
print(f"   🌐 VPN templates: {len(vpn_templates['templates'])}")
print(f"   🆔 Identity templates: {len(identity_templates['templates'])}")
print(f"   🏢 Enterprise templates: {len(enterprise_templates['templates'])}")
print(f"   📈 Total new templates: {len(security_templates['templates']) + len(monitoring_templates['templates'])}")